# 3.0 - Evaluate models on the test set

In [1]:
import os
import torch
import json
from transformers import AutoModelForCausalLM, AutoTokenizer
import numpy as np

# 1. Configuration
MODEL_ID = "Qwen/Qwen3-0.6B"
#MODEL_ID = "Qwen/Qwen3-1.7B"
TESTDATA_FILE = os.path.join(os.getcwd(), '..', 'data', 'train_test_dataset', 'orange_qa_test.jsonl')

# 2. Load Model & Tokenizer
print(f"Loading {MODEL_ID}...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    dtype=torch.float16,       # Use float16 to save memory
    device_map="auto",          # Auto-selects GPU or CPU
    do_sample=False,
)


/Users/martin/Documents/FRI/Workshops/LoRA-tutorial/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading Qwen/Qwen3-0.6B...


In [2]:
# 3. Load Data
with open(TESTDATA_FILE, "r") as f:
    test_dataset = json.load(f)

In [3]:
import sys
sys.path.append(os.path.join(os.getcwd(), '..'))

from src.evaluation_function import evaluate_model

acc, SE = evaluate_model(model, tokenizer, test_dataset)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


In [4]:
acc, SE

(np.float64(7.0), np.float64(1.8041618552668715))